In [ ]:
#!pip install webdriver-manager

# Youtube link

# https://www.youtube.com/watch?v=_AeudsbKYG8

In [1]:
import csv
from bs4 import BeautifulSoup

# Webdriver for chrome
from selenium import webdriver
##from webdriver_manager.chrome import ChromeDriverManager

In [5]:
#driver = webdriver.Chrome(ChromeDriverManager().install())
driver = webdriver.Chrome()

In [6]:
def get_url(search_term):
    """Generate a url from search term"""
    template = 'https://www.amazon.com/s?k={}'
    search_term = search_term.replace(' ', '+')
    return template.format(search_term)


In [7]:
search_term = input()
url = get_url(search_term)
print(url)

books
https://www.amazon.com/s?k=books


## Extraction of the data

In [15]:
driver.get('https://www.amazon.com/s?k=books')

WebDriverException: Message: chrome not reachable
  (Session info: chrome=100.0.4896.60)
Stacktrace:
Backtrace:
	Ordinal0 [0x00C97413+2389011]
	Ordinal0 [0x00C29F61+1941345]
	Ordinal0 [0x00B1C520+836896]
	Ordinal0 [0x00B10682+788098]
	Ordinal0 [0x00B10EB8+790200]
	Ordinal0 [0x00B12752+796498]
	Ordinal0 [0x00B0C0D9+770265]
	Ordinal0 [0x00B1D9D0+842192]
	Ordinal0 [0x00B73AE2+1194722]
	Ordinal0 [0x00B63F66+1130342]
	Ordinal0 [0x00B3E546+976198]
	Ordinal0 [0x00B3F456+980054]
	GetHandleVerifier [0x00E49632+1727522]
	GetHandleVerifier [0x00EFBA4D+2457661]
	GetHandleVerifier [0x00D2EB81+569713]
	GetHandleVerifier [0x00D2DD76+566118]
	Ordinal0 [0x00C30B2B+1968939]
	Ordinal0 [0x00C35988+1989000]
	Ordinal0 [0x00C35A75+1989237]
	Ordinal0 [0x00C3ECB1+2026673]
	BaseThreadInitThunk [0x768C6739+25]
	RtlGetFullPathName_UEx [0x77DB8E7F+1215]
	RtlGetFullPathName_UEx [0x77DB8E4D+1165]


In [8]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [9]:
results = soup.find_all('div', {'data-component-type' : 's-search-result'} )

In [10]:
len(results)

0

## Prototype data of a single record

In [12]:
item = results[0]

IndexError: list index out of range

In [11]:
atag = item.h2.a

NameError: name 'item' is not defined

In [58]:
description = atag.text.strip()
description

'2021 Newest HP 14" HD Laptop Light-Weight, AMD 3020e(Up to 2.6GHz), 8GB RAM, 128GB SSD + 64GB eMMC, 1 Year Office 365, WiFi, Bluetooth 5, USB Type-A&C, HDMI, Webcam, Win10, w/Ghost Manta Accessories'

In [59]:
url = 'https://www.amazon.com' + atag.get('href')
url

'https://www.amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A0010730H206M7DG367P&url=%2FNewest-HP-Light-Weight-Bluetooth-Accessories%2Fdp%2FB093PPMDKZ%2Fref%3Dsr_1_1_sspa%3Fkeywords%3Dlaptops%26qid%3D1649092327%26sr%3D8-1-spons%26psc%3D1&qualifier=1649092327&id=805029717201978&widgetName=sp_atf'

In [60]:
price_parent = item.find('span','a-price')
price = price_parent.find('span', 'a-offscreen').text
price

'$399.99'

In [61]:
rating = item.i.text
rating

'4.3 out of 5 stars'

In [65]:
review_count = item.find('span', {'class': 'a-size-base s-underline-text'}).text
review_count

'821'

# Final Data Extract

In [18]:
import csv
from bs4 import BeautifulSoup

# Webdriver for chrome
from selenium import webdriver



def get_url(search_term):
    """Generate a url from search term"""
    template = 'https://www.amazon.com/s?k={}'
    search_term = search_term.replace(' ', '+')
    
    # Add term query to url
    url = template.format(search_term)
    
    # add the page query placeholder
    url += '&page{}'
    
    return url
   

def extract_record(item):
    """Extract and return ddadta from a single record"""
    
    # description and url
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://amazon.com' + atag.get('href')
    
    try:
        # Price
        price_parent = item.find('span','a-price')
        price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return
    
    try:
        # Rank and rating
        rating = item.i.text
        review_count = item.find('span', {'class': 'a-size-base s-underline-text'}).text
    except AttributeError:
        rating = ''
        review_count=''

        
    result = ( description, price, rating, review_count, url)
    
    return result

def main(search_term):
    """Run main program routine"""
    # startup the webdriver
    driver = webdriver.Chrome()
    
    record = []
    url = get_url(search_term)
    
    for page in range(1, 21):
        driver = url.format(page)
        soup = BeautifulSoup(driver, 'html.parser')
        results = soup.find_all('div', {'data-component-type' : 's-search-result'} )
        
        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)
                
        
    #driver.close()
    
    # Save daa to csv file
    with open('results.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description', 'Price', 'Rating', 'ReviewCount', 'Url'])
        writer.writerows(records)
    

In [19]:
main('iphonex')

NameError: name 'records' is not defined